Loading the dataset to perform 10-fold cross-validation evaluation

In [1]:
from Utils.LoadDataset import Dataloader

dataset  = Dataloader(data_path = "./Custom Dataset/Raw Data",
                  labels_path = "./Custom Dataset/labels.csv", 
                  seperator = ",", test_ratio = 0.2, validation_ratio = 0.2,
                  classnames=['running', 'walking', 'sitting', 'lying'])

Creating the stratified folds for splitting the dataset into 10 folds.

In [2]:
# Creating fold for training.

from sklearn.model_selection import KFold,StratifiedKFold
stratifiedKfold = StratifiedKFold(n_splits = 10, random_state = None, shuffle = True)

Creating dictionaries for original model and the tflite model to access the precision, recall and f1-score of every class in the dataset.

In [3]:
from collections import namedtuple, defaultdict

model_metrics = defaultdict(dict)
tflite_model_metrics = defaultdict(dict)
metric_names = ["precision", "recall", "f1-score"]

for classname in dataset.classnames:
    for metric in metric_names:
        model_metrics[classname][metric] = 0.0
        tflite_model_metrics[classname][metric] = 0.0

perfroming 10-fold cross-validation

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import sklearn
from sklearn.model_selection import KFold,StratifiedKFold
from Utils.Optimization import OptimizeModel

fold_no = 0

tf.random.set_seed(1)

for train, test in stratifiedKfold.split(dataset.all.x, dataset.all.y):

    model = keras.Sequential()
    model.add(keras.Input(shape = (50, 1, 3), name = "accelartion"))
    model.add(keras.layers.Conv2D(filters = 14, kernel_size = (5, 1), padding = "valid", activation = "relu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Conv2D(filters = len(dataset.classnames), kernel_size = (5, 1), padding = "same", 
                                  activation = "relu"))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.MaxPool2D((46, 1)))
    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(len(dataset.classnames), use_bias=True))

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy()])
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    model.fit(dataset.all.x[train], dataset.all.y[train],
                    batch_size=64,
                    epochs=100,
                    verbose=0)
    
    model_predictions = model.predict(dataset.all.x[test])
    model_labels = np.argmax(model_predictions, axis = -1)
    
    model_classification_report = sklearn.metrics.classification_report((dataset.all.y[test]), 
                                                                        model_labels,
                                                                        target_names = (dataset.classnames), 
                                                                        digits = 4, 
                                                                        output_dict = True)
    
    for activity_name, metric_scores in model_classification_report.items():
        
        if activity_name in dataset.classnames:       
            for metric, value in metric_scores.items():
                
                if metric in metric_names:             
                    model_metrics[activity_name][metric] += model_classification_report[activity_name][metric]
                else:
                    continue
        else:
            continue
    
    tflite_converted_model = OptimizeModel(model, dataset, quantize = True, assertion = True)
    tflite_model_predictions = tflite_converted_model.predict(dataset.all.x[test])
    tflite_model_labels = np.argmax(tflite_model_predictions, axis = -1)
    
    tflite_model_classification_report = sklearn.metrics.classification_report((dataset.all.y[test]), 
                                                                        tflite_model_labels,
                                                                        target_names = (dataset.classnames), 
                                                                        digits = 4, 
                                                                        output_dict = True)
    
    for activity_name, metric_scores in tflite_model_classification_report.items():
        
        if activity_name in dataset.classnames:       
            for metric, value in metric_scores.items():
                
                if metric in metric_names:             
                    tflite_model_metrics[activity_name][metric] += tflite_model_classification_report[activity_name][metric]
                else:
                    continue
        else:
            continue
    
    fold_no = fold_no + 1
    

------------------------------------------------------------------------
Training for fold 0 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmphuk9j6qn\assets
------------------------------------------------------------------------
Training for fold 1 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmp647es0d4\assets


INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmp647es0d4\assets


------------------------------------------------------------------------
Training for fold 2 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmp7ngylz45\assets


INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmp7ngylz45\assets


------------------------------------------------------------------------
Training for fold 3 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmpdrogqbcl\assets


...

INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmpbze21zf1\assets


------------------------------------------------------------------------
Training for fold 8 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmpzpfdjg_q\assets


INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmpzpfdjg_q\assets


------------------------------------------------------------------------
Training for fold 9 ...
INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmptt881i7g\assets


INFO:tensorflow:Assets written to: C:\Users\sande\AppData\Local\Temp\tmptt881i7g\assets


The performance estimates of all the classes evaluated using 10-fold cross-validation.

In [8]:
model_metrics

defaultdict(dict,
            {'running': {'precision': 9.950546448087431,
              'recall': 9.933333333333334,
              'f1-score': 9.941594323679885},
             'walking': {'precision': 9.950273224043714,
              'recall': 9.932768361581921,
              'f1-score': 9.941020768523995},
             'sitting': {'precision': 9.817106025555926,
              'recall': 8.948305084745764,
              'f1-score': 9.35378958077727},
             'lying': {'precision': 9.103856065594645,
              'recall': 9.916666666666668,
              'f1-score': 9.487594657252275}})

In [9]:
tflite_model_metrics

defaultdict(dict,
            {'running': {'precision': 9.918288383571303,
              'recall': 9.933333333333334,
              'f1-score': 9.925200881056934},
             'walking': {'precision': 9.950273224043714,
              'recall': 9.916101694915254,
              'f1-score': 9.932617407179457},
             'sitting': {'precision': 9.720866597145088,
              'recall': 9.01497175141243,
              'f1-score': 9.344892792836},
             'lying': {'precision': 9.147228541137483,
              'recall': 9.8,
              'f1-score': 9.456909463485024}})